In [1]:
#Script to evaluate the determinants of patent citation 
#Renato Kogeyama

# Apr 27, 2020
# Separe most central patents, classify them as disruptive and calculate dvs from extant literature

# Mar 16, 2020
# Introducing centrality measures as dv

# Feb 04, 2020
# to set labels in heatmap keyword:xticklabels
# for ex.
# sns.heatmap(globalWarming_df, xticklabels = np.arange(0,15))
# to be implemented later
# another alternative is to substitute the values in the dataset and convert columns to categories
# to understand the impact, i should run some test
# however i am focusing now in calculate Corredoira's 2015 and Nemet & Johnson 2012

# Feb 03, 2020
# version backed up as _old

# Feb 02, 2020
# the best way to deal with the classification names is to use a dictionary
# this avoid charging memory with the strings
# However, WIPO is organized differently than the other systems
# I'll update the wipo code to uniformize the behavior in this script
# I am creating a code that reflects the first level of classification 

# Feb 01, 2020
# Introduction of categorical graphs: barplot and heatmap
# heatmap is not the real deal, its a simplification
# the real deal would be the correlation table - there is a suggestion based on cramer, 
    # but implementation was not ready
# graphs exported and google docs updated
# next step: update cit_tree to reflect Corredoira's 205 Influence measure
# plot a network graph: https://plot.ly/python/network-graphs/
# reproduce 2012 Nemet and Johnson with other class systems
# correct bias in generality and originality (multiply for N/N-1)

# Jan 21, 2020
# Classifications added
# Code reorganized - much faster now
# Still missing the update of applications to the grant number
# I should provide now descriptive statistics on all variables

# Jan 21, 2020
# The current data does not have Class
# I should go back and get this info - but there are too many scripts now and
#   I should reorganize them before moving forward
# I should also include the patent publication date - to control for the policy changes
# In the citation file, I should change application number for grant when possible 
#   This will improve realiability of all measures related to citation
# Introduce classifications

# Jan 18, 2020
# Variables calculated
# Generality, average delay, forward and backward citations, cumulative citation (cit_tree)
# Still missing originality
# the file with variables that are used in this script should get a name independent from the date


#Miami, December 24th, 2019
# Prof. Rafael Corredoira suggested:
# - Inclusion of a tree of citations
#   To track back the source of citations. This is information is not given by direct count of citations.
# - Consider policy changes in the way patents are cited
#   Policy changes in 2000 changed the time frame of citation, and 2010 partially moved citation to applications
# - Track classification changes 
#   The original classification system in USPTO changed from a technical based to a market based classification system
#   See if there is an impact
# - Consider a text analysis of the claims
#   Classification is based on the claims but it is not clear how many claims are related to each classification category
# - Include moderation effect from classification
#   Citations patterns may change across industries, so some effects may disappear if industry is not accounted for.

# In summary, his ideas help increase structure of the current work


#Syracuse, December 3rd, 2019

#The original script is getting too complex
#There was many tentative scripts to play with data
#Here I am writing a script to show the relevance of variables to patent citation

#11-12-2019
#Introducing normalization

#10-11-2019
#I introduced log backward citation, what corrects for very dispersed results
#but the major problem is that few patents receive citations
#bring back binary output

#10-10-2019
#Added graphics and new distributions

#10-03-2019
#I rewrote the citation data to clean the strings

#09-15-2019
#O naive bayes tem algum problema com distribuicoes desbalanceadas
#o scikit learn tem um modulo que corrige count distributions com muitos zeros, o complementNB
#porem este nao esta disponivel na atual versao disponibilizada no HPC da FIU

#09-10-2019
#o trabalho pede uma abordagem mais sistematica e cuidadosa
#estou agrupando o codigo antigo comentado e vou comecar um novo codigo

#09-27-2019
#I am renaming citation as forward citation and backward citation

#09-17-2018

#Alto uso de memoria - rodar no Amazon AWS 



In [2]:
import pandas as pd
import numpy as np
import IPython.display as display
import seaborn as sns
          
import itertools

from sklearn import preprocessing
from sklearn import linear_model, datasets
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn import naive_bayes
from sklearn.metrics import roc_curve, auc
from sklearn.feature_selection import VarianceThreshold

import scipy.stats as ss
import datetime
import matplotlib.pyplot as plt

from math import sqrt

import sys
sys.path.append('/home/rkogeyam/scripts/')
sys.path.append('scripts/')

from determinants_scripts import classes

from plotbar import plotbar
from plot_heat import heatmap


from best_num_attr import best_num_attr
from xattrSelect import xattrSelect
from sampler import sampler
from normalize import normalize
from nbayes import nbayes

import gzip

In [3]:
latex='data/results.tex'
# dataset='data/dataset.csv'
dataset=gzip.open('data/dataset.csv.gz', 'rt')

In [4]:
%matplotlib inline
sns.set()
sns.set_palette(sns.cubehelix_palette(8))
# pd.options.display.float_format = '{:,.2f}'.format

In [5]:
dtypes={'id':object,'type':object, 'kind':object, 'num_claims':float, 'cit_received':float, 'cit_made':float,
       'cit_received_delay':float, 'cit_made_delay':float, 'parent_citation':float,
       'originality':float, 'generality':float, 'wipo_sector_id':object, 'ipcr_section':object,
       'ipcr_ipc_class':object, 'ipcr_subclass':object, 'cpc_section_id':object,
       'cpc_subsection_id':object, 'cpc_group_id':object, 'nber_category_id':object,
       'nber_subcategory_id':object, 'uspc_mainclass_id':object, 'uspc_subclass_id':object, 'eigen':float, 'pagerank':float, 'katz':float}

In [6]:
# # only main classes (exclude uspc)

# usecols=['id', 'type', 'date', 'kind', 'num_claims', 'cit_received', 'cit_made',
#          'cit_received_delay', 'cit_made_delay', 'parent_citation',
#          'originality', 'generality', 'wipo_sector_id', 'ipcr_section', 
#          'cpc_section_id', 'nber_category_id', 'eigen', 'pagerank', 'katz']

In [7]:
# only WIPO class system, exclude type and kind

usecols=['id', 'date', 'num_claims', 'cit_received', 'cit_made',
         'cit_received_delay', 'cit_made_delay', 'parent_citation',
         'originality', 'generality', 'wipo_sector_id', 'eigen', 'pagerank', 'katz']

In [8]:
df=pd.read_csv(dataset, dtype=dtypes, usecols=usecols, parse_dates=['date'], index_col='id')

# df.info()

In [9]:
df['year']=df.date.dt.year

df['decade']=df.date.dt.year//10*10
df['decade'] =df['decade'].apply(lambda x: int(x) if str(x) != 'nan' else np.nan)
decades=list(df.decade.unique())
# decades = [int(x) for x in decades if str(x) != 'nan']

In [10]:
obj_cols=list(df.select_dtypes(include=[object]).columns.values)
obj_cols

['wipo_sector_id']

In [11]:
num_cols=list(df.select_dtypes(include=[np.number]).columns.values)
num_cols

['num_claims',
 'cit_received',
 'cit_made',
 'cit_received_delay',
 'cit_made_delay',
 'parent_citation',
 'originality',
 'generality',
 'pagerank',
 'katz',
 'eigen',
 'year',
 'decade']

## Data Analysis

### Descriptive 

In [12]:
descriptive=df.describe(include=[np.number]).loc[['count','mean','std','min','max']].append(df[num_cols].isnull().sum().rename('isnull'))

/home/applications/anaconda3/4.2.0/lib/python3.5/site-packages/numpy/lib/function_base.py:4269: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


In [13]:
descriptive.apply(lambda x: x.apply('{:,.2f}'.format)).transpose()

,count,mean,std,min,max,isnull
num_claims,"6,488,254.00",14.14,12.00,0.00,887.00,"2,214,063.00"
cit_received,"8,702,317.00",10.51,27.95,0.00,"3,922.00",0.00
cit_made,"6,183,715.00",14.78,41.95,0.00,"5,811.00","2,518,602.00"
cit_received_delay,"7,046,552.00",20.44,24.36,-285.83,207.58,"1,655,765.00"
cit_made_delay,"6,175,204.00",13.68,10.69,-280.52,182.74,"2,527,113.00"
parent_citation,"6,256,483.00",247.30,"1,588.33",1.00,"227,479.00","2,445,834.00"
originality,"5,604,693.00",0.15,0.21,0.00,0.80,"3,097,624.00"
generality,"6,552,762.00",0.15,0.21,0.00,0.80,"2,149,555.00"
pagerank,"8,519,764.00",0.00,0.00,0.00,0.00,"182,553.00"
katz,"8,519,764.00",0.00,0.00,0.00,0.02,"182,553.00"


In [14]:
df.describe(include=[np.object])#.append(df[np.object].isnull().sum().rename('isnull')).transpose()

,wipo_sector_id
count,5879781
unique,5
top,1
freq,2158667


### Barplots and Heatmaps

In [15]:
# # barplot
# # as of 02.03.20, working

# for i in obj_cols:
#     plotbar(i, df, classes)

# # barplot with decades
# for i in obj_cols:
#     plotbar(i, df, classes,decade=True)

# # barplot with decades and inverted axis
# for i in obj_cols:
#     plotbar(i, df, classes,decade=True, decade_x=True)

# # heatmaps all periods
# for double in list(itertools.combinations(obj_cols, 2)):
#     heatmap(df[double[0]], df[double[1]]) 

# # print heatmaps per decade
# for decade in decades:
#     df_dec=df[df['decade']==decade]
#     for double in list(itertools.combinations(obj_cols, 2)):
#         heatmap(df_dec[double[0]], df_dec[double[1]], decade) 


### Histograms

In [16]:
# #histograms
# #could improve cutting off outliers
# for variable in num_cols:
#     ax=df[variable].hist()
#     ax.set_title('Histogram '+ variable.title()+'\n')
#     plt.show()

### Trends and Boxplots

In [17]:
#iterate over numerical variables

num_cols.remove('decade')
num_cols.remove('year')

In [18]:
# for variable in num_cols:
    
#     title=variable.replace('_', ' ')
#     fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))

#     axes[0] = df.groupby('year').mean().plot(y=variable, ax=axes[0])
#     evl_title='Evolution of '+ title +'\n'
#     axes[0].set_title(evl_title)
#     axes[0].set_ylim(bottom=0)
    
#     axes[1] = sns.boxplot(x='decade', y=variable, data=df)

#     box_title='Dispersion of '+ title +'\n'
#     axes[1].set_title(box_title)
#     axes[1].set_ylim(bottom=0)
#     axes[1].set_ylabel("")
    
#     filename='./img/evol_dispersion_'+variable.lower()+'.png'  
#     plt.savefig(filename) 
#     plt.show()


In [19]:
# the generality data on the 2010's is too concentrated around 0
# to check, I draw this hist to understand what is happening
# it could be an effect of truncation - generality increases with forward citation

# df[df['decade']==2010]['generality'].hist()

### Models

In [20]:
# normalization
df=normalize(df.dropna())

Total number of observations and attributes
(3315219, 15)
Number of numerical attributes: 13
Number of non-numerical attributes: 1


/home/rkogeyam/scripts/normalize.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[num_cols]=min_max_scaler.fit_transform(df[num_cols])
/home/applications/anaconda3/4.2.0/lib/python3.5/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/home/rkogeyam/scripts/normalize.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata

In [21]:
# #maybe nb fit does not accept nomalized data, so i using data without normalize
# #but in that case, i have to transform the categorical variables

# obj_cols=list(df.select_dtypes(include=[object]).columns.values)

# for col in obj_cols:
#     df[col] = df[col].astype('category')

# df=pd.get_dummies(df, columns=obj_cols, prefix=obj_cols)

In [22]:
# List of IVs
chosenColumns=df.columns.values.tolist()
len(chosenColumns)

19

In [23]:
chosenColumns.remove('cit_received')
len(chosenColumns)

18

In [24]:
chosenColumns.remove('parent_citation')
len(chosenColumns)

17

In [25]:
chosenColumns.remove('katz')
len(chosenColumns)

16

In [26]:
chosenColumns.remove('eigen')
len(chosenColumns)

15

In [27]:
chosenColumns.remove('pagerank')
len(chosenColumns)

14

In [28]:
chosenColumns.remove('date')
len(chosenColumns)

13

### DV: citation received (forward citation)

In [29]:
# myX = df.as_matrix(columns=chosenColumns)
# myY = df.as_matrix(columns=['cit_received'])

# xTrain, xTest, yTrain, yTest = train_test_split(myX, myY, train_size=0.7, random_state=3) 
# testSize = yTest.shape[0]
# trainSize = yTrain.shape[0]

/home/applications/anaconda3/4.2.0/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [30]:
# namesList, errorList = best_num_attr(myX, xTrain, xTest, yTrain, yTest, chosenColumns, regtype='linear')

Out of sample error versus attribute set size
  cit_received_delay 0.01186264801084925
          num_claims 0.011712056820717317
    wipo_sector_id_2 0.011644875546558612
    wipo_sector_id_1 0.011552265695752562
            cit_made 0.011507837780579806
                year 0.011479830619280939
      cit_made_delay 0.011446475788336487
    wipo_sector_id_4 0.011443970056290664
              decade 0.011443626443882773
         originality 0.01144344215261313
    wipo_sector_id_0 0.011443376319769119
          generality 0.011443373876874316
    wipo_sector_id_3 0.011443373253911954


### DV: Parent citation (rename the variable)

In [31]:
# myX = df.as_matrix(columns=chosenColumns)

# myY = df.as_matrix(columns=['parent_citation'])

# xTrain, xTest, yTrain, yTest = train_test_split(myX, myY, train_size=0.7, random_state=3) 
# testSize = yTest.shape[0]
# trainSize = yTrain.shape[0]
# namesList, errorList = best_num_attr(myX, xTrain, xTest, yTrain, yTest, chosenColumns, regtype='linear')

/home/applications/anaconda3/4.2.0/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Out of sample error versus attribute set size
  cit_received_delay 0.009064909379753676
    wipo_sector_id_2 0.009017693943675109
    wipo_sector_id_1 0.008991999197410696
          num_claims 0.008978955175629405
            cit_made 0.008969849274179304
      cit_made_delay 0.008964828034852195
          generality 0.008960913134240864
                year 0.008959849789264954
    wipo_sector_id_4 0.008959074911425823
              decade 0.008958840717204224
         originality 0.008958683821017603
    wipo_sector_id_0 0.008958681575683398
    wipo_sector_id_3 0.008958681273422907


### DV: pagerank (centrality)

In [32]:
# myX = df.as_matrix(columns=chosenColumns)
# myY = df.as_matrix(columns=['pagerank'])

# xTrain, xTest, yTrain, yTest = train_test_split(myX, myY, train_size=0.7, random_state=3) 
# testSize = yTest.shape[0]
# trainSize = yTrain.shape[0]

/home/applications/anaconda3/4.2.0/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [33]:
# namesList, errorList = best_num_attr(myX, xTrain, xTest, yTrain, yTest, chosenColumns, regtype='linear')

Out of sample error versus attribute set size
                year 0.004831920226149007
    wipo_sector_id_1 0.004759807351203704
      cit_made_delay 0.004743937272376311
    wipo_sector_id_2 0.004733411972073784
          num_claims 0.004723887978914528
  cit_received_delay 0.004722376719855053
         originality 0.0047220255689645555
    wipo_sector_id_0 0.004721727827304055
            cit_made 0.00472147516317153
    wipo_sector_id_4 0.004721369990883928
          generality 0.0047213434903858546
              decade 0.004721324289381794
    wipo_sector_id_3 0.004721324311245939


### DV: katz (centrality)

In [34]:
# myX = df.as_matrix(columns=chosenColumns)
# myY = df.as_matrix(columns=['katz'])

# xTrain, xTest, yTrain, yTest = train_test_split(myX, myY, train_size=0.7, random_state=3) 
# testSize = yTest.shape[0]
# trainSize = yTrain.shape[0]

/home/applications/anaconda3/4.2.0/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [35]:
# namesList, errorList = best_num_attr(myX, xTrain, xTest, yTrain, yTest, chosenColumns, regtype='linear')

Out of sample error versus attribute set size
  cit_received_delay 0.008777641162395081
          num_claims 0.008716064667651017
    wipo_sector_id_2 0.008664666456597449
    wipo_sector_id_1 0.008619696622148862
            cit_made 0.008598371434101224
      cit_made_delay 0.008586582850287914
                year 0.008574086318915338
    wipo_sector_id_4 0.00857289058205384
          generality 0.008571904705226127
              decade 0.008571662539183903
         originality 0.008571561728862977
    wipo_sector_id_0 0.00857155150048133
    wipo_sector_id_3 0.008571547146729962


### DV: eigen (centrality)

In [36]:
# myX = df.as_matrix(columns=chosenColumns)
# myY = df.as_matrix(columns=['eigen'])

# xTrain, xTest, yTrain, yTest = train_test_split(myX, myY, train_size=0.7, random_state=3) 
# testSize = yTest.shape[0]
# trainSize = yTrain.shape[0]

/home/applications/anaconda3/4.2.0/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [37]:
# namesList, errorList = best_num_attr(myX, xTrain, xTest, yTrain, yTest, chosenColumns, regtype='linear')

Out of sample error versus attribute set size
                year 0.0012808355445282854
    wipo_sector_id_1 0.0012808229463154115
  cit_received_delay 0.0012808173968997122
      cit_made_delay 0.0012808168862736432
    wipo_sector_id_2 0.00128081630709277
            cit_made 0.0012808161669796557
    wipo_sector_id_4 0.0012808161051043449
    wipo_sector_id_0 0.0012808160920165879
    wipo_sector_id_3 0.0012808160945770304
         originality 0.0012808161045346825
          num_claims 0.001280816156687945
          generality 0.0012808166152320844
              decade 0.0012808191766125012


In [38]:
# This selector does not work because almost every attribute is p-value significant

# selector = SelectKBest(f_classif, k=4) #initialize 
# selector.fit(myX, myY) #fit
# scores = -np.log10(selector.pvalues_) #transform pvalues (why?)
# scores /= scores.max() #normalize 
# plt.bar(myX - .45, scores, width=.2,
#         label=r'Univariate score ($-Log(p_{value})$)', color='darkorange',
#         edgecolor='black')

In [39]:
# nbayes(xTrain, yTrain, xTest, yTest)

In [ ]:
df.dropna()

In [40]:
#Let's do something else
#Change the DV 

myX = df.as_matrix(columns=chosenColumns)
myY = df.as_matrix(columns=['parent_back_citation'])

xTrain, xTest, yTrain, yTest = train_test_split(myX, myY, train_size=0.7, random_state=3) 

In [41]:
nbayes(xTrain, yTrain, xTest, yTest)

In [42]:
# df.parent_back_citation.boxplot()

In [43]:
#and graphs of back citation in time

In [44]:
# for i in classifications:
#     rank=df.groupby(i).count().iloc[:,2].sort_values(ascending=False).reset_index().set_index(i)
#     description=df_class[df_class['class']==i].set_index('id')
#     display(rank.join(description))

In [45]:
# for i in obj_cols:
#     if i.isin(classifications):
#         df.join(df.groupby(i).count().iloc[:,2].sort_values(ascending=False)
# #     display.display(df.pivot_table(values=df.reset_index().id, index=i, columns='decade', aggfunc='count', fill_value=0, margins=False, dropna=True))
#     print(i)
#     display.display(df.groupby(i).count().iloc[:,2].sort_values(ascending=False))

In [46]:
# def cramers_v(x, y):
#     confusion_matrix = pd.crosstab(x,y)
#     chi2 = ss.chi2_contingency(confusion_matrix)[0]
#     n = confusion_matrix.sum().sum()
#     phi2 = chi2/n
#     r,k = confusion_matrix.shape
#     phi2corr = max(0, phi2-((k-1)*(r-1))/(n-1))
#     rcorr = r-((r-1)**2)/(n-1)
#     kcorr = k-((k-1)**2)/(n-1)
#     return np.sqrt(phi2corr/min((kcorr-1),(rcorr-1)))

In [47]:
# all variables
# dtypes={'id':object, 'type':object, 'kind':object, 'num_claims':float, 'cit_received':float, 'cit_made':float,
#        'cit_received_delay':float, 'cit_made_delay':float, 'parent_citation':float,
#        'originality':float, 'generality':float, 'wipo_field_id':object, 'ipcr_section':object,
#        'cpc_section_id':object,'nber_category_id':object,'uspc_mainclass_id':object}